<div dir="center">
  <h1 align="center" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      پیشنهاد سرعتی
    </font>
  </h1>
</div>

<p dir="rtl" style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
  <font face="vazirmatn" size="3">
  در این تمرین قرار است الگوریتم <code>Funk-SVD</code> را با استفاده از گرادیان کاهشی تصادفی قدم به قدم از مرحله 0 پیاده‌سازی کنیم. سپس به کمک آن، یک مدل پیش‌بینی را آموزش دهیم که به سرعت بتواند علایق یک کاربر را پیش‌بینی کند و به سرعت لیست پیشنهادی برای کاربر آماده کند.
  </font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    در این تمرین قرار است از ماتریس‌های تعاملی که در درس‌نامه قبل ساختیم استفاده کنیم. این ماتریس‌ها در پوشه <code>Data</code> ذخیره شده‌اند. به ترتیب ماتریس موجود در فایل <code>train_matrix.npz</code> را در متغیر <code>train</code> و ماتریس موجود در <code>test_matrix.npz</code> را در متغیر <code>test</code> بارگذاری و ذخیره کنید.

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    همچنین بردار آیدی‌های منحصر به فرد کاربران و آیتم‌ها به ترتیب با کلیدهای <code>users</code> و <code>items</code> در فایل <code>unique.npz</code> در پوشه <code>Data</code> ذخیره شده است. آن‌ها را به ترتیب در متغیرهای <code>unique_user_ids</code> و <code>unique_item_ids</code> ذخیره کنید.
</font>
</p>


<div dir="rtl">
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
<font color="red"><b color='red'>نکته مهم:</b></font>

ماتریس‌ها را با استفاده از متد `scipy.sparse.load_npz` بخوانید تا در ادامه برای اجرای کدها به مشکل برنخورید.
</font>
</p>
</div>

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
import scipy.sparse
import random


train_matrix_path = '../Data/train_matrix.npz'
test_matrix_path = '../Data/test_matrix.npz'
unique_ids_path = '../Data/unique.npz'

# TODO
train = scipy.sparse.load_npz(train_matrix_path)
test = scipy.sparse.load_npz(test_matrix_path)

unique_ids = np.load(unique_ids_path, allow_pickle=True)
unique_user_ids = unique_ids['users']
unique_item_ids = unique_ids['items']

In [2]:
unique_user_ids

array([     8,      9,     12, ..., 262128, 262130, 262138],
      shape=(64078,))

In [3]:
unique_user_ids.shape

(64078,)

<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام اول
    </font>
  </h2>
</div>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazirmatn; font-size:medium">
<font face="vazirmatn" size=3>
  برای ساخت ماتریس‌های ویژگی‌های پنهان کاربر (P) و ماتریس ویژگی پنهان آیتم (Q) به تعداد کاربران و آیتم‌ها نیاز داریم. به ترتیب در متغیرهای <code>num_users</code> و <code>num_items</code> تعداد کاربران و آیتم‌ها را ذخیره کنید. برای این‌کار می‌توانید از ابعاد ماتریس <code>train</code> کمک بگیرید.
</font>
</p>


In [4]:
# TODO
num_users, num_items = train.shape

print(f"Number of unique users: {num_users}")
print(f"Number of unique items: {num_items}")

Number of unique users: 64078
Number of unique items: 144804


<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام دوم:  تعریف پارامترهای مدل و الگوریتم
    </font>
  </h2>
</div>




<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
  در این بخش پارامترهای مدل یادگیری را تعریف و به میزان مناسب مقداردهی کنید.
  توضیحات پارامترها به شرح زیر می‌باشد:


</font>
</p>



<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>k</code>|تعداد ویژگی‌های پنهان|
|<code>learning_rate</code>|نرخ یادگیری|
|<code>lambda_reg</code>| ترم منظم‌سازی |
|<code>num_epochs</code>|تعداد دوره‌های یادگیری|
|<code>log_step</code>|مشخص می‌کند که در حین یادگیری، هر چند دوره یک بار مقدار ارور محاسبه شود|


</font>
</div>
</center>


<div dir="rtl">
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
<font color="green"><b color='green'>پیشنهاد:</b></font>

با پارامترهای کوچک، مثل `k=50` و `num_epochs=5`، شروع کنید و در صورتی که نتیجه مطلوب نبود، این مقادیر را تغییر دهید تا به بهتری نتیجه برسید.
</font>
</p>
</div>

In [17]:
# TODO
k = 100
learning_rate = 0.01
lambda_reg = 0.06
num_epochs = 10
log_step = 1

print(f"Hyperparameters: k={k}, learning_rate={learning_rate}, lambda_reg={lambda_reg}, num_epochs={num_epochs}, log_step={log_step}")

Hyperparameters: k=100, learning_rate=0.01, lambda_reg=0.06, num_epochs=10, log_step=1


<div dir="rtl">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام سوم: مقادیردهی اولیه پارامترهای مدل
    </font>
  </h2>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
  در این بخش باید دو ماتریس P و Q را بسازید و با مقادیر تصادفی و کوچک مقداردهی کنید. ابعاد این ۲ ماتریس را با کمک مقادیر  <code>num_users</code>، <code>num_items</code> و <code>k</code> که در گام‌های اول و دوم مشخص شدند، تعیین کنید.

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
  در قدم بعد برای یادگیری سوگیری‌های کاربران و آیتم‌ها، ۲ آرایه تماما 0 به اندازه تعداد کاربران و آیتم‌ها در متغیرهای مشخص شده تعریف کنید.
</font>
</p>


<div dir="rtl">
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
  در نهایت برای پیاده‌سازی الگوریتم توسعه‌یافته Funk-SVD نیاز است که میانگین کلی تمام امتیازهای ماتریس آموزش را محاسبه کنید. این مقدار را در متغیر <code>global_average</code> ذخیره کنید.

<details>
<summary>
راهنمایی
</summary>
برای به دست آوردن مقادیر ماتریس می‌توانید و <code>train.data</code> کمک بگیرید.
</details>

</div>



In [18]:
rng = np.random.default_rng(seed=42)
init_std = 0.01

In [19]:
rng

Generator(PCG64) at 0x796E1A7FE500

In [20]:
# TODO
P = rng.normal(scale=init_std, size=(num_users, k))
Q = rng.normal(scale=init_std, size=(k, num_items))

user_bias = np.zeros(num_users)
item_bias = np.zeros(num_items)

global_average = train.data.mean()


print(f"Initialized P shape: {P.shape}")
print(f"Initialized Q shape: {Q.shape}")
print(f"Initialized user_bias shape: {user_bias.shape}")
print(f"Initialized item_bias shape: {item_bias.shape}")
print(f"Calculated global average: {global_average:.4f}")

Initialized P shape: (64078, 100)
Initialized Q shape: (100, 144804)
Initialized user_bias shape: (64078,)
Initialized item_bias shape: (144804,)
Calculated global average: 7.6299


<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام چهارم: نوشتن توابع کمکی
    </font>
  </h2>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
در این بخش برای پیاده‌سازی الگوریتم یادگیری، به تعدادی توابع کمکی نیاز داریم که در هر قسمت توضیحات توابع آورده شده است. با نوشتن این توابع، می‌توانیم به صورت ماژولار الگوریتم را پیاده‌سازی کنیم تا در ادامه بتوانیم به راحتی کدمان را دیباگ کنیم.
</font>
</p>

<div dir="right">
  <h4 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تابع پیش‌بینی امتیاز
    </font>
  </h4>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
بدنه تابع <code>calculate_predicted_rating</code> را طوری بنویسید که با گرفتن مقادیر مناسب، مقدار امتیاز کاربر <code>u</code> به آیتم <code>i</code> را پیش‌بینی کند. برای پیش‌بینی باید مقدار زیر محاسبه شود:

$$PredictedRating_{[u, i]} = P_{[u, :]} \cdot Q_{[:, i]} + GlobalAverage + UserBias_{[u]} + ItemBias_{[i]}$$
</font>
</p>

In [21]:
def calculate_predicted_rating(user_idx, item_idx, P, Q, user_bias, item_bias, global_average):
    """
    Calculates the predicted rating for a single user-item pair.

    Args:
        user_idx (int): Mapped index of the user.
        item_idx (int): Mapped index of the item.
        P (np.ndarray): User latent feature matrix (num_users, k).
        Q (np.ndarray): Item latent feature matrix (k, num_items).
        user_bias (np.ndarray): User bias vector (num_users,).
        item_bias (np.ndarray): Item bias vector (num_items,).
        global_average (float): Global average rating.

    Returns:
        float: The predicted rating.
    """
    # TODO
    predicted_rating = (
        P[user_idx, :] @ Q[:, item_idx]
        + global_average
        + user_bias[user_idx]
        + item_bias[item_idx]
    )

    return predicted_rating

<div dir="right">
  <h4 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تابع محاسبه خطا
    </font>
  </h4>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
بدنه تابع <code>calculate_error</code> را طوری بنویسید که با گرفتن مقادیر مناسب، مقدار خطای امتیاز پیش‌بینی‌شده کاربر <code>u</code> به آیتم <code>i</code> را محاسبه کند. برای محاسبه خطا باید مقدار زیر محاسبه شود:

$$Error = ActualRating - PredictedRating$$
</font>
</p>

In [22]:
def calculate_error(actual_rating, predicted_rating):
    """
    Calculates the prediction error for a single rating.

    Args:
        actual_rating (float): The actual observed rating.
        predicted_rating (float): The predicted rating.

    Returns:
        float: The prediction error (actual - predicted).
    """
    # TODO
    error = actual_rating - predicted_rating

    return error

<div dir="right">
  <h4 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تابع به‌روزرسانی سوگیری‌ها
    </font>
  </h4>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
بدنه تابع <code>update_biases</code> را طوری بنویسید که با گرفتن مقادیر مناسب سوگیری‌های کاربرها و آیتم‌ها را به‌روز‌رسانی کند. برای به‌روزرسانی مقادیر آرایه بایاس مورد نظر، به شرح زیر عمل کنید.

$$BiasArray_{[i]} \leftarrow LearningRate * (Error - \lambda * BiasArray_{[i]})$$
</font>
</p>

In [23]:
def update_biases(user_idx, item_idx, error, user_bias, item_bias, learning_rate, lambda_reg):
    """
    Updates the user and item biases using the calculated error and SGD rules.

    Args:
        user_idx (int): Mapped index of the user.
        item_idx (int): Mapped index of the item.
        error (float): The prediction error for the user-item pair.
        user_bias (np.ndarray): User bias vector (num_users,).
        item_bias (np.ndarray): Item bias vector (num_items,).
        learning_rate (float): The learning rate for SGD.
        lambda_reg (float): The regularization parameter.
    """

    # TODO
    user_bias[user_idx] += learning_rate * (error - lambda_reg * user_bias[user_idx])
    item_bias[item_idx] += learning_rate * (error - lambda_reg * item_bias[item_idx])

<div dir="right">
  <h4 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تابع به‌روزرسانی مقادیر ویژگی‌های پنهان
    </font>
  </h4>
</div>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
بدنه تابع <code>update_latent_features</code> را طوری بنویسید که مقادیر ویژگی‌های پنهان کاربرها و آیتم‌ها را به‌روز‌رسانی کند. این تابع به عنوان ورودی اندیس کاربر، اندیس آیتم، میزان خطای پیش‌بینی شده، ماتریس‌های P و Q به همراه پارامترهای مدل را می‌گیرد و برای به‌روزرسانی مقادیر ماتریس‌های P و Q به ازای هر عنصر f در بردارهای ویژگی‌های پنهان کاربر و آیتم به ترتیب به روش زیر عمل می‌کند:

$$P_{uf} \leftarrow P_{uf} + \alpha \left(  \cdot \text{error} \cdot Q_{fi} -  \lambda P_{uf} \right)$$


$$Q_{fi} \leftarrow Q_{fi} + \alpha \left( \text{error} \cdot P_{uf} - \lambda Q_{fi} \right)$$

</font>
</p>

In [24]:
def update_latent_features(user_idx, item_idx, error, P, Q, learning_rate, lambda_reg, k):
    """
    Updates the user and item latent features using the calculated error and SGD rules.

    Args:
        user_idx (int): Mapped index of the user.
        item_idx (int): Mapped index of the item.
        error (float): The prediction error for the user-item pair.
        P (np.ndarray): User latent feature matrix (num_users, k).
        Q (np.ndarray): Item latent feature matrix (k, num_items).
        learning_rate (float): The learning rate for SGD.
        lambda_reg (float): The regularization parameter.
        k (int): The number of latent features.
    """

    for f in range(k):
        # TODO
        P[user_idx, f] += learning_rate * (error * Q[f, item_idx] - lambda_reg * P[user_idx, f])
        Q[f, item_idx] += learning_rate * (error * P[user_idx, f] - lambda_reg * Q[f, item_idx])


<div dir="right">
  <h4 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تابع محاسبه خطا
    </font>
  </h4>
</div>

<div dir="rtl">
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>

به عنوان آخرین تابع کمکی، نیاز به تابعی برای محاسبه خطای *Root Mean Squared Error* داریم. این تابع به ما برای ارزیابی مدل آموزش‌دیده برای مسئله پیش‌بینی امتیاز کمک ویژه‌ای می‌کند.

تابع زیر را به‌گونه‌ای بنویسید:

- با فراخوانی تابع `calculate_predicted_rating` امتیاز کاربر $u$ به آیتم $i$ را پیش‌بینی کند.

- میزان خطای این پیش‌بینی را با کم‌کردن این مقدار از امتیاز واقعی محاسبه کند. سپس توان دوم این خطا را به متغیر `sse` اضافه کند.

- در نهایت با کمک `sse` مقدار **ریشه دوم میانگین خطا** یا `rmse` را برگرداند.

</font>
</p>

In [25]:
def calculate_rmse(P, Q, user_bias, item_bias, global_average, sparse_matrix):
    """
    Calculates the Root Mean Squared Error (RMSE) on a sparse matrix.

    Args:
        P (np.ndarray): Trained user latent feature matrix.
        Q (np.ndarray): Trained item latent feature matrix.
        user_bias (np.ndarray): Trained user bias vector.
        item_bias (np.ndarray): Trained item bias vector.
        global_average (float): Calculated global average rating.
        sparse_matrix (matrix): The sparse matrix to evaluate on (e.g., test set).

    Returns:
        float: The calculated RMSE.
    """

    rows = sparse_matrix.row
    cols = sparse_matrix.col
    data = sparse_matrix.data
    num_samples = len(data)

    if num_samples == 0:
        return 0.0 # Avoid division by zero

    sse = 0 # Sum of Squared Errors
    for idx in range(num_samples):
        # TODO
        u = rows[idx]
        i = cols[idx]
        r_ui = data[idx]

        predicted_rating = calculate_predicted_rating(u, i, P, Q, user_bias, item_bias, global_average)
        error = calculate_error(r_ui, predicted_rating)
        sse += error ** 2



    rmse = np.sqrt(sse / num_samples)
    return rmse


<div dir="rtl">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      گام پنجم: پیاده‌سازی حلقه آموزش
    </font>
  </h2>
</div>


<p dir=rtl style="line-height:200%; font-size:medium">
<font face="vazirmatn" size=4>
در این بخش به پیاده‌سازی حلقه اصلی آموزش می‌پردازیم. ابتدا نیاز است که برای عملیات‌های سطری و ستونی، سطرها و ستون‌ها را در متغیرهای جدا ذخیره کنید. با استفاده از این سطرها و ستون‌ها، می‌توانیم به مقادیر مرتبط با هرکاربر و هرآیتم به راحتی دسترسی داشته باشیم.
</font>
</p>


<div dir="rtl">
<p style="line-height:200%; font-size:medium">
<font face="vazirmatn" size=4>

**برای پیاده‌سازی حلقه اصلی آموزش، بهتر است روش زیر عمل کنیم.**


برای هر `epoch` از ۱ تا `num_epochs`:  


- اندیس‌های داده‌های آموزشی را به صورت تصادفی (`shuffle`) مرتب کنید.  

 - برای هر نمونه `idx` در داده‌های تصادفی‌شده:  

	- کاربر (`u`)، آیتم (`i`)، و رتبه واقعی (`r_ui`) را از داده‌های آموزشی استخراج کنید.  
       
    -  با استفاده از تابع کمکی `calculate_predicted_rating`، امتیاز کاربر `u` به آیتم `i` را پیش‌بینی کنید.
      
    - با استفاده از تابع کمکی`calculate_error`، میزان خطای پیش‌بینی را محاسبه کنید.
    
    - با استفاده از تابع کمکی `update_biases` سوگیری‌های کاربران و آیتم‌ها را به‌روزرسانی کنید.
     
    - با استفاده از تابع کمکی `update_latent_features` مقادیر ویژگی‌های پنهان در ماتریس‌های P و Q به‌روزرسانی کنید.


سپس در ادامه میزان `rmse` را در هر `log_step` دوره، محاسبه و چاپ می‌کنیم تا میزان همگرایی مدل را ببینیم.


<details>
<summary>نکته</summary>
تنها قسمت مشخص‌شده با کامنت TODO را تکمیل کنید.
</details>
</font>
</p>
</div>


In [26]:
train_rows = train.row
train_cols = train.col
train_data = train.data
num_train_samples = len(train_data)

print("Starting training...")

for epoch in range(num_epochs):
    # Shuffle the order of training samples
    shuffled_indices = list(range(num_train_samples))
    random.shuffle(shuffled_indices)

    # Iterate through shuffled training samples
    for idx in shuffled_indices:
        # TODO
        u = train_rows[idx]
        i = train_cols[idx]
        r_ui = train_data[idx]
        
        predicted_rating = calculate_predicted_rating(u, i, P, Q, user_bias, item_bias, global_average)
        error = calculate_error(r_ui, predicted_rating)
        update_biases(u, i, error, user_bias, item_bias, learning_rate, lambda_reg)
        update_latent_features(u, i, error, P, Q, learning_rate, lambda_reg, k)

    # Calculate and print training RMSE after this epoch (or periodically)
    if (epoch + 1) % log_step == 0:
        train_rmse = calculate_rmse(P, Q, user_bias, item_bias, global_average, train)
        test_rmse = calculate_rmse(P, Q, user_bias, item_bias, global_average, test)
        print(f"Epoch {epoch+1}/{num_epochs}, Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}")

print("\nTraining finished.")

Starting training...
Epoch 1/10, Train RMSE: 1.6738, Test RMSE: 1.6977
Epoch 2/10, Train RMSE: 1.6199, Test RMSE: 1.6676
Epoch 3/10, Train RMSE: 1.5822, Test RMSE: 1.6512
Epoch 4/10, Train RMSE: 1.5519, Test RMSE: 1.6403
Epoch 5/10, Train RMSE: 1.5263, Test RMSE: 1.6328
Epoch 6/10, Train RMSE: 1.5039, Test RMSE: 1.6276
Epoch 7/10, Train RMSE: 1.4830, Test RMSE: 1.6232
Epoch 8/10, Train RMSE: 1.4636, Test RMSE: 1.6202
Epoch 9/10, Train RMSE: 1.4446, Test RMSE: 1.6181
Epoch 10/10, Train RMSE: 1.4249, Test RMSE: 1.6160

Training finished.


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
  در این قسمت نیاز است که عملکرد مدل خود را به طور همزمان بر روی ماتریس آموزش و آزمون در نظر بگیرید. اگر عملکرد مدل شما بر روی ماتریس آزمون مورد قبول نیست، برای مثال در این‌جا میزان خطای بالای 1.7، بهتر است با بازتنظیم پارامترهای مدل، به بهبود مدل یادگیری خود بپردازید. در ضمن توجه کنید که میزان خطای سیستم برروی داده‌های آموزش و آزمون، نباید فرق قابل توجهی داشته باشند!
</font>
</p>

<div dir="rtl">
<p dir=rtl style="line-height:200%; font-size:medium">
<font face="vazirmatn" size=4>
حال که آموزش مدل به پایان رسید، می‌توانید با استفاده از کد زیر، <b>میزان امتیازی که کاربر مورد نظر به یک آیتم خاص می‌دهد را محاسبه کنید</b>.

برای این کار کافی‌ست این متغیرهای `user_idx` و `item_idx` را با مقادیر مناسب پر کنید. در ادامه با استفاده از کدی مشابه به سلول پایین، عملیات پیشنهاد دادن را انجام دهیم.
</font>
</p>
</div>

In [27]:
unique_user_ids

array([     8,      9,     12, ..., 262128, 262130, 262138],
      shape=(64078,))

In [28]:
unique_item_ids

array(['0002005018', '074322678X', '0887841740', ..., '0883966336',
       '0935906010', '1880461048'], shape=(144804,), dtype=object)

In [31]:
user_idx = 8
item_idx = 100


try:
    mapped_user_idx = np.where(unique_user_ids == user_idx)[0][0]
    mapped_item_idx = unique_item_ids[item_idx]
    mapped_item_idx_for_prediction = np.where(unique_item_ids == mapped_item_idx)[0][0]


    predicted_rating_example = calculate_predicted_rating(mapped_user_idx, mapped_item_idx_for_prediction, P, Q, user_bias, item_bias, global_average)
    print(f"\nPredicted rating for original user ID {user_idx} (mapped index {mapped_user_idx}) and original item ID {unique_item_ids[item_idx]} (mapped index {mapped_item_idx_for_prediction}): {predicted_rating_example:.4f}")

except IndexError:
    print("\nCould not find mapped index for example prediction. Ensure original IDs exist in unique IDs.")


Predicted rating for original user ID 8 (mapped index 0) and original item ID 0671021745 (mapped index 100): 6.4098


<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      تشکیل لیست پیشنهادی
    </font>
  </h2>
</div>

<div dir="rtl">
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>

در این قسمت تابع <code>get_recommendations</code> برای شما از قبل پیاده‌سازی شده است تا با کمک آن و مدلی که آموزش داده‌اید، برای کاربر دلخواه یک لیست پیشنهادی از کتاب‌ها به طول دلخواه <code>n</code> بسازید. این تابع امتیازهای کاربر به آیتم‌ها را پیش‌بینی می‌کند؛ سپس `n` آیتم اول با بیشترین امتیاز را برمی‌گرداند.

این تابع با گرفتن مقادیر مناسب به ترتیب به صورت زیر عمل می‌کند:

- بررسی می‌کند که آیدی کاربر مورد نظر معتبر باشد.
- آیتم‌هایی را که کاربر امتیازی برای آن‌ها ثبت نکرده را پیدا می‌کند.
- امتیاز کاربر به این آیتم‌ها را پیش‌بینی می‌کند.
- با کمک مرتب‌سازی لیست امتیازهای پیش‌بینی شده، `n` آیتم اولی که بیشترین امتیاز برای آن‌ها پیش‌بینی شده را پیدا می کند.
- آیدی آیتم‌های پیشنهادی را برمی‌گرداند.
</font>
</p>
<div>

In [46]:
user_idx = 8
rated_items = train.getrow(user_idx).indices

print(rated_items)

train.data

[20 19]


array([9, 6, 9, ..., 5, 7, 8], shape=(290827,))

In [ ]:
def get_recommendations(original_user_id, n, P, Q, user_bias, item_bias, global_average, train_matrix, unique_user_ids, unique_item_ids):
    """
    Generates top N recommendations for a given user.

    Args:
        original_user_id: The original ID of the user.
        n (int): The number of recommendations to generate.
        P (np.ndarray): Trained user latent feature matrix (num_users, k).
        Q (np.ndarray): Trained item latent feature matrix (k, num_items).
        user_bias (np.ndarray): Trained user bias vector (num_users,).
        item_bias (np.ndarray): Trained item bias vector (num_items,).
        global_average (float): Calculated global average rating.
        train_matrix (matrix): The sparse training interaction matrix.
        unique_user_ids (np.ndarray): Array mapping mapped user indices to original IDs.
        unique_item_ids (np.ndarray): Array mapping mapped item indices to original IDs.

    Returns:n_matrix
        list: A list of tuples (original_item_id, predicted_rating) for the top N recommendations.
              Returns an empty list if the user ID is not found.
    """
    # Find the mapped index for the original user ID
    try:
        user_idx = np.where(unique_user_ids == original_user_id)[0][0]
    except IndexError:
        print(f"Warning: Original user ID {original_user_id} not found in unique user IDs.")
        return [] # Return empty list if user not found

    num_items = Q.shape[1]
    all_item_indices = np.arange(num_items)

    # Find items the user has already rated in the training set
    user_row_sparse = train_matrix.getrow(user_idx)
    rated_item_indices = user_row_sparse.indices

    # Find items the user has NOT rated
    unrated_item_indices = np.setdiff1d(all_item_indices, rated_item_indices)

    user_latent_features = P[user_idx, :] # Shape (k,)
    unrated_item_latent_features = Q[:, unrated_item_indices] # Shape (k, num_unrated_items)

    # Calculate dot products: result is (num_unrated_items,)
    latent_predictions = user_latent_features @ unrated_item_latent_features # Using @ for matrix multiplication (1xk @ kxN -> 1xN)

    predicted_ratings_for_user = global_average + user_bias[user_idx] + item_bias[unrated_item_indices] + latent_predictions


    # Get the indices of the top N predicted ratings within the unrated_item_indices array
    top_n_indices_in_unrated = np.argsort(predicted_ratings_for_user)[::-1][:n]

    # Get the mapped item indices of the top N recommended items
    recommended_mapped_item_indices = unrated_item_indices[top_n_indices_in_unrated]

    # Get the original item IDs and their predicted ratings
    recommended_items_with_ratings = [(unique_item_ids[item_idx], predicted_ratings_for_user[i])
                                      for i, item_idx in enumerate(recommended_mapped_item_indices)] # Use index 'i' from top_n_indices_in_unrated to get the correct predicted rating

    return recommended_items_with_ratings


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
برای استفاده از تابع <code>get_recommendations</code> کافی است از قطعه کد زیر استفاده کنید و متغیر <code>original_user_id_to_recommend</code> را با آیدی اصلی کاربر مورد نظر و <code>top_n_recommendations</code> را با یک عدد صحیح مثبت که نشان‌دهنده تعداد آیتم‌های پیشنهادی است مقداردهی کنید.
</font>
</p>

<style>
.rtl {
  direction: rtl;
  text-align: right;
  font-family: vazir;
}
</style>

<div class="rtl">
<details class="green">
<summary>
توجه
</summary>
برای استفاده بهتر می‌توانید آید‌ی‌های معتبر کاربران را از لیست <code>unique_user_ids</code> به دست بیاورید.
</details>
</div>

In [ ]:
original_user_id_to_recommend = None
top_n_recommendations = None

recommendations = get_recommendations(original_user_id_to_recommend, top_n_recommendations, P, Q, user_bias, item_bias, global_average, train, unique_user_ids, unique_item_ids)

if recommendations:
    print(f"\nTop {top_n_recommendations} recommendations for original user ID {original_user_id_to_recommend}:")
    for item_id, predicted_rating in recommendations:
        print(f"  Item ID: {item_id}, Predicted Rating: {predicted_rating:.4f}")

<div dir="right">
  <h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
    <font face="vazirmatn" color="#0099cc">
      ایجاد پاسخ
    </font>
  </h2>
</div>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
در سلول زیر لیستی از آیدی‌های کاربران نوشته شده است. با اجرای کد زیر و با کمک تابع <code>get_recommendations</code> لیستی پیشنهادی برای کاربران مورد نظر ساخته می‌شود. عملکرد شما در این تمرین با کمک این لیست سنجیده خواهد شد. از تغییر محتویات سلول زیر اجتناب کنید.
</font>
</p>

In [ ]:
import pandas as pd

eval_users_idx = [8, 9, 12, 14, 16, 17, 19, 22, 26, 32, 39, 42, 44, 51, 53, 56, 64, 67, 69, 70]
eval_top_n_recommendations = 10
eval_recommendations = [get_recommendations(user_idx, eval_top_n_recommendations, P, Q, user_bias, item_bias, global_average, train, unique_user_ids, unique_item_ids)
                        for user_idx in eval_users_idx]

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نوت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [ ]:
import zipfile
import os
import scipy.sparse
import numpy as np

if not os.path.exists(os.path.join(os.getcwd(), 'initial.ipynb')):
    %notebook -e initial.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)


np.savez('submission.npz', submission=eval_recommendations)


file_names = ['initial.ipynb', 'submission.npz']
compress(file_names)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
ثبت دیدگاه
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
امیدواریم از انجام این تمرین لذت برده باشید. دعوت می‌کنیم در صفحه تمرین نظر خود را در رابطه با این روش‌ تجزیه ماتریسی و تفاوت آن با روش‌های قبلی را با بقیه کاربران به اشتراک بگذارید.
</font>
</p>